In [1]:
import os
%pwd

'/Users/kuchibhotlaanirudharaghavasarma/textsummarizer/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/kuchibhotlaanirudharaghavasarma/textsummarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        # Create ModelTrainerConfig instance
        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/kuchibhotlaanirudharaghavasarma/textsummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-21 00:13:12,884: INFO: config: PyTorch version 2.5.1 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        try:
            # Force CPU usage
            device = "cpu"

            # Load model and tokenizer
            tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail", cache_dir="./model_cache")
            model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail", cache_dir="./model_cache").to(device)

            # Enable gradient checkpointing
            model_pegasus.gradient_checkpointing_enable()

            # Data collator
            seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

            # Load datasets (use subsets for testing)
            dataset = load_from_disk(self.config.data_path)
            train_dataset = dataset["train"].select(range(5))
            eval_dataset = dataset["validation"].select(range(2))

            # Training arguments
            trainer_args = TrainingArguments(
                output_dir=self.config.root_dir,
                num_train_epochs=1,
                warmup_steps=100,
                per_device_train_batch_size=1,
                per_device_eval_batch_size=1,
                weight_decay=0.01,
                logging_steps=10,
                evaluation_strategy="steps",
                eval_steps=50,
                save_steps=50,
                gradient_accumulation_steps=8,
            )

            # Trainer initialization
            trainer = Trainer(
                model=model_pegasus,
                args=trainer_args,
                tokenizer=tokenizer,
                data_collator=seq2seq_data_collator,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
            )

            # Train model
            trainer.train()

            # Save model and tokenizer
            model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
            tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

        except Exception as e:
            print(f"Training failed: {e}")


In [9]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
Found existing installation: accelerate 1.2.1
Uninstalling accelerate-1.2.1:
  Successfully uninstalled accelerate-1.2.1
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
Using cached accelerate-1.2.1-py3-none-any.whl (336 kB)


In [10]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2024-12-21 00:13:19,566: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-21 00:13:19,569: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-21 00:13:19,571: INFO: common: created directory at: artifacts]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/kuchibhotlaanirudharaghavasarma/textsummarizer/venv/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/gq/82j4wg7x5cv5287zbycvh_2c0000gn/T/ipykernel_60426/252967937.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
100%|██████████| 1/1 [00:06<00:00,  6.42s/it]/Users/kuchibhotlaanirudharaghavasarma/textsummarizer/venv/lib/python3.10/site-packages/transform

{'train_runtime': 15.0116, 'train_samples_per_second': 0.333, 'train_steps_per_second': 0.067, 'train_loss': 16.8542537689209, 'epoch': 1.0}
